# DS/CMPSC 410 Spring 2022
# Instructor: Professor John Yen
# TA: Rupesh Prajapati 
# LA's: Lily Jakielaszek and Cayla Shan Pun

# Lab 4: Data Frames Transformation, SQL Functions, and Join
# The goals of this lab are for you to be able to
## - Use Data Frames in Spark for Processing Structured Data
## - Perform Basic DataFrame Transformation: Filtering Rows and Selecting Columns of DataFrame
## - Create New Column of DataFrame using `withColumn`
## - Use SQL Function split, col 
## - Filter on a Column that is an Array using `array_contains`
## - Transform DataFrame to RDD, and RDD to DataFrame
## - Perform Join on DataFrames 
## - Perform sorting on a DataFrame column
## - Generate histogram from an RDD
## - Apply the obove to find Movies in a Genre that has good reviews with a significant number of ratings.

## Total Number of Exercises: 
- Exercise 1: 5 points
- Exercise 2: 5 points
- Exercise 3: 10 points
- Exercise 4: 10 points
- Exercise 5: 10 points
- Exercise 6: 10 points
- Exercise 7: 10 points
- Exercise 8: 10 points
- Exercise 9: 10 points
- Exercise 10: 10 points
- Exercise 11: 10 points
## Total Points: 100 points

# Due: midnight, February 6, 2022

## The first thing we need to do in each Jupyter Notebook running pyspark is to import pyspark first.

In [1]:
import pyspark

### Once we import pyspark, we need to import "SparkContext".  Every spark program needs a SparkContext object
### In order to use Spark SQL on DataFrames, we also need to import SparkSession from PySpark.SQL

In [2]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField, StructType, StringType, LongType, IntegerType, FloatType
from pyspark.sql.functions import col, column
from pyspark.sql.functions import expr
from pyspark.sql.functions import split
from pyspark.sql import Row

## We then create a Spark Session variable (rather than Spark Context) in order to use DataFrame. 
- Note: We temporarily use "local" as the parameter for master in this notebook so that we can test it in jupyter Server.  However, we need to remove `.master("local")` before we submit it to ICDS cluster to run this in cluster  mode.

In [3]:
ss=SparkSession.builder.master("local").appName("Lab 4 DataFrame").getOrCreate()

# Exercise 1 (5 points) 
- (a) Add your name below AND 
- (b) replace the path below in both `ss.read.csv` statements with the path of your home directory.

## Answer for Exercise 1 (Double click this Markdown cell to fill your name below.)
- a: Student Name: Haichen Wei

In [4]:
movies_DF = ss.read.csv("/storage/home/hxw5245/Lab4/movies_2.csv", header=True, inferSchema=True)

In [5]:
movies_DF.printSchema()

root
 |-- MovieID: integer (nullable = true)
 |-- MovieTitle: string (nullable = true)
 |-- Genres: string (nullable = true)



In [6]:
movies_DF.show(10)

+-------+--------------------+--------------------+
|MovieID|          MovieTitle|              Genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
|      6|         Heat (1995)|Action|Crime|Thri...|
|      7|      Sabrina (1995)|      Comedy|Romance|
|      8| Tom and Huck (1995)|  Adventure|Children|
|      9| Sudden Death (1995)|              Action|
|     10|    GoldenEye (1995)|Action|Adventure|...|
+-------+--------------------+--------------------+
only showing top 10 rows



In [7]:
ratings_DF = ss.read.csv("/storage/home/hxw5245/Lab4/ratings_2.csv", header=True, inferSchema=True)

In [8]:
ratings_DF.printSchema()

root
 |-- UserID: integer (nullable = true)
 |-- MovieID: integer (nullable = true)
 |-- Rating: double (nullable = true)
 |-- RatingID: integer (nullable = true)



In [9]:
ratings_DF.show(5)

+------+-------+------+----------+
|UserID|MovieID|Rating|  RatingID|
+------+-------+------+----------+
|     1|     31|   2.5|1260759144|
|     1|   1029|   3.0|1260759179|
|     1|   1061|   3.0|1260759182|
|     1|   1129|   2.0|1260759185|
|     1|   1172|   4.0|1260759205|
+------+-------+------+----------+
only showing top 5 rows



# 2. DataFrames Transformations
DataFrame in Spark provides higher-level transformations that are convenient for selecting rows, columns, and for creating new columns.  These transformations are part of Spark SQL.

## 2.1 DF Transformation for Filtering Rows
Select rows from a DataFrame that satisfy a condition.  This is similar to "WHERE" clause in SQL query language.

In [10]:
movies_DF.where(col("MovieTitle")== "Toy Story (1995)").show()

+-------+----------------+--------------------+
|MovieID|      MovieTitle|              Genres|
+-------+----------------+--------------------+
|      1|Toy Story (1995)|Adventure|Animati...|
+-------+----------------+--------------------+



In [11]:
movies_DF.where("Toy Story (1995)" == col("MovieTitle")).show()

+-------+----------------+--------------------+
|MovieID|      MovieTitle|              Genres|
+-------+----------------+--------------------+
|      1|Toy Story (1995)|Adventure|Animati...|
+-------+----------------+--------------------+



In [12]:
ratings_DF.where(col("Rating") > 3).count()

62106

In [13]:
ratings_DF.filter(3 < col("Rating")).count()

62106

# Exercise 2 (5 points) Filtering DF Rows
### Complete the following statement to (1) select the `ratings_DF` DataFrame for reviews that are exactly 5, and (2) count the total number of such reviews.

In [14]:
ratings_DF.where( col("Rating") == 5 ).count()

15095

## 2.2 DataFrame Transformation for Selecting Columns

DataFrame transformation `select` is similar to the projection operation in SQL: it returns a DataFrame that contains all of the columns selected.

In [15]:
movies_DF.select("MovieTitle").show(5)

+--------------------+
|          MovieTitle|
+--------------------+
|    Toy Story (1995)|
|      Jumanji (1995)|
|Grumpier Old Men ...|
|Waiting to Exhale...|
|Father of the Bri...|
+--------------------+
only showing top 5 rows



In [16]:
movies_DF.select(col("MovieTitle")).show(5)

+--------------------+
|          MovieTitle|
+--------------------+
|    Toy Story (1995)|
|      Jumanji (1995)|
|Grumpier Old Men ...|
|Waiting to Exhale...|
|Father of the Bri...|
+--------------------+
only showing top 5 rows



# Exercise 3 (10 points) Selecting DF Columns
## Complete the following PySpark statement to (1) select only `MovieID` and `Rating` columns, and (2) save it in a DataFrame called `movie_rating_DF`.

In [17]:
movie_rating_DF = ratings_DF.select("MovieID", "Rating")

In [18]:
movie_rating_DF.show(5)

+-------+------+
|MovieID|Rating|
+-------+------+
|     31|   2.5|
|   1029|   3.0|
|   1061|   3.0|
|   1129|   2.0|
|   1172|   4.0|
+-------+------+
only showing top 5 rows



# 2.3 Statistical Summary of Numerical Columns
DataFrame provides a `describe` method that provides a summary of basic statistical information (e.g., count, mean, standard deviation, min, max) for numerical columns.

In [19]:
ratings_DF.describe().show()

+-------+------------------+------------------+------------------+--------------------+
|summary|            UserID|           MovieID|            Rating|            RatingID|
+-------+------------------+------------------+------------------+--------------------+
|  count|            100004|            100004|            100004|              100004|
|   mean| 347.0113095476181|12548.664363425463| 3.543608255669773|1.1296390869392424E9|
| stddev|195.16383797819535|26369.198968815268|1.0580641091070326|1.9168582602710962E8|
|    min|                 1|                 1|               0.5|           789652009|
|    max|               671|            163949|               5.0|          1476640644|
+-------+------------------+------------------+------------------+--------------------+



## RDD has a histogram method to compute the total number of rows in each "bucket".
The code below selects the Rating column from `ratings_DF`, converts it to an RDD, which maps to extract the rating value for each row, which is used to compute the total number of reviews in 6 buckets. For example, the first bucket is for reviews >= 0, but less than 1. The last bucket is
for reviews >=5, but less than 6.

In [20]:
ratings_DF.select(col("Rating")).rdd.map(lambda row: row[0]).histogram([0,1,2,3,4,5,6])

([0, 1, 2, 3, 4, 5, 6], [1101, 5013, 11720, 30602, 36473, 15095])

# 3. Transforming the Generes Column into Array of Generes 
## We want transform a column Generes, which represent all Generes of a movie using a string that uses "|" to connect the Generes so that we can later filter for movies of a Genere more efficiently.
## This transformation can be done using `split` Spark SQL function (which is different from python `split` function)

In [21]:
Splitted_Generes_DF= movies_DF.select(split(col("Genres"), '\|'))
Splitted_Generes_DF.show(5)

+---------------------+
|split(Genres, \|, -1)|
+---------------------+
| [Adventure, Anima...|
| [Adventure, Child...|
|    [Comedy, Romance]|
| [Comedy, Drama, R...|
|             [Comedy]|
+---------------------+
only showing top 5 rows



## 3.1 Adding a Column to a DataFrame using withColumn
### We often need to transform content of a column into another column. For example, we would like to transform the column Genres in the movies DataFrame into an `Array` of genres that each movie belongs, we can do this using the DataFrame method `withColumn`.

# Exercise 4 (10 points)
Complete the code below to create a new column called "Genres_Array", whose values are arrays of genres for each movie.

In [22]:
movies2_DF= movies_DF.withColumn("Genres_Array",split("Genres", '\|') )

In [23]:
movies2_DF.printSchema()

root
 |-- MovieID: integer (nullable = true)
 |-- MovieTitle: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Genres_Array: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [24]:
movies2_DF.show(5)

+-------+--------------------+--------------------+--------------------+
|MovieID|          MovieTitle|              Genres|        Genres_Array|
+-------+--------------------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|[Adventure, Anima...|
|      2|      Jumanji (1995)|Adventure|Childre...|[Adventure, Child...|
|      3|Grumpier Old Men ...|      Comedy|Romance|   [Comedy, Romance]|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|[Comedy, Drama, R...|
|      5|Father of the Bri...|              Comedy|            [Comedy]|
+-------+--------------------+--------------------+--------------------+
only showing top 5 rows



# 4. Computing Total Reviews and Average Rating for Each Movie
Because it is convenient and efficient to compute both total reviews and average rating for each movie using key value pairs, we will convert the reviews Data Frame into RDD.

In [25]:
ratings_RDD = ratings_DF.rdd
ratings_RDD.take(3)

[Row(UserID=1, MovieID=31, Rating=2.5, RatingID=1260759144),
 Row(UserID=1, MovieID=1029, Rating=3.0, RatingID=1260759179),
 Row(UserID=1, MovieID=1061, Rating=3.0, RatingID=1260759182)]

In [26]:
movie_ratings_RDD = ratings_RDD.map(lambda row: (row.MovieID, row.Rating))

In [27]:
movie_ratings_RDD.take(10)

[(31, 2.5),
 (1029, 3.0),
 (1061, 3.0),
 (1129, 2.0),
 (1172, 4.0),
 (1263, 2.0),
 (1287, 2.0),
 (1293, 2.0),
 (1339, 3.5),
 (1343, 2.0)]

# Exercise 5 (10 points)
Complete the code below to compute the total number of reviews for each movie.

In [28]:
movie_review_1_RDD = movie_ratings_RDD.map(lambda x: (x[0] , 1 ))
movie_review_total_RDD = movie_review_1_RDD.reduceByKey(lambda x, y: x+y , 3)

In [29]:
movie_review_total_RDD.take(10)

[(1029, 42),
 (1263, 48),
 (1287, 46),
 (1293, 46),
 (1371, 47),
 (1953, 46),
 (2193, 42),
 (39, 120),
 (144, 26),
 (150, 200)]

# Exercise 6 (10 points)
Complete the code below to compute the total ratings for each movie.

In [30]:
# Compute average rating for each movie
rating_total_RDD = movie_ratings_RDD.reduceByKey(lambda x, y: x+y, 3)

In [31]:
rating_total_RDD.take(10)

[(1029, 155.5),
 (1263, 185.5),
 (1287, 179.0),
 (1293, 183.0),
 (1371, 143.5),
 (1953, 185.0),
 (2193, 134.5),
 (39, 426.0),
 (144, 86.5),
 (150, 780.5)]

## Join Transformation on Two RDDs
Two Key Value Pairs RDDs can be joined on the RDD (similar to the join operation in SQL) to return a new RDD, whose rows is an inner join of the two input RDDs.  Only key value pairs occur in both input RDDs occur in the output RDD.

In [32]:
# Join the two RDDs (one counts total reviews, the other computes sum of ratings)
joined_RDD = rating_total_RDD.join(movie_review_total_RDD)

In [33]:
joined_RDD.take(4)

[(1029, (155.5, 42)),
 (1263, (185.5, 48)),
 (1287, (179.0, 46)),
 (1293, (183.0, 46))]

# The following code computes average rating for each movie from the joined RDD.

In [34]:
# Compute average rating for each movie
average_rating_RDD = joined_RDD.map(lambda x: (x[0], x[1][0]/x[1][1] ))

In [35]:
average_rating_RDD.take(4)

[(1029, 3.7023809523809526),
 (1263, 3.8645833333333335),
 (1287, 3.891304347826087),
 (1293, 3.9782608695652173)]

### The following code joins the average_rating_RDD with movie_review_total_RDD so that we obtain an RDD in the form of 
```
(<movieID>, (<average rating>, <total review>)
```
### because we want to keep both average rating and total review of each movie so that we can filter on either of them.

In [36]:
# We want to keep both average review and total number of reviews for each movie. 
# So we do another join her.
avg_rating_total_review_RDD = average_rating_RDD.join(movie_review_total_RDD)

In [37]:
avg_rating_total_review_RDD.take(4)

[(144, (3.326923076923077, 26)),
 (150, (3.9025, 200)),
 (168, (3.116279069767442, 43)),
 (186, (2.875, 48))]

## Transforming RDD to Data Frame
An RDD can be transformed to a Data Frame using toDF().  We want to transform the RDD containing average rating and total reviews for each movie into a Data Frame so that we can answer questions that involve both movie reviews and generes such as the following:
- What movies in a genre (e.g., comedy) has a top 10 average review among those that receive at least k reviews?

In [38]:
# Before transforming to Data Frame, we first convert the key value pairs of avg_rating_total_reivew_RDD 
# which has the format of (<movie ID> (<average rating> <review total>) )  to a tuple of the format
# (<movie ID> <average rating> <review total>)
avg_rating_total_review_tuple_RDD = avg_rating_total_review_RDD.map(lambda x: (x[0], x[1][0], x[1][1]) )

In [39]:
avg_rating_total_review_tuple_RDD.take(5)

[(144, 3.326923076923077, 26),
 (150, 3.9025, 200),
 (168, 3.116279069767442, 43),
 (186, 2.875, 48),
 (222, 3.9318181818181817, 22)]

## Defining a Schema for Data Frame
As we have seen before, each Data Frame has a Schema, which defines the names of the column and the type of values for the column (e.g., string, integer, or float).  There are two ways to specify the schema of a Data Frame:
- Infer the schema from the heading and the value of an input file (e.g., CSV).  This is how the schema of movies_DF was created in the beginning of this notebook.
- Explicitly specify the Schema
We will use one approach in the second category here to specify the column names and the type of column values of the DataFrame to be converted from the RDD above.

# Exercise 7 (10 points)
Define a schema and use it to convert the `avg_rating_total_reive_tuple_RDD` to a DataFrame. The schema should contains three columns: MovieID, AvgRating, and TotalReviews with type integer, float, and integer respectively.

In [40]:
schema = StructType([ StructField("MovieID", IntegerType(), True ), \
                     StructField("AvgRating", FloatType(), True ), \
                     StructField("TotalReviews", IntegerType(), True) \
                    ])

In [41]:
# Convert the RDD to a Data Frame
avg_review_DF = avg_rating_total_review_tuple_RDD.toDF(schema)

In [42]:
avg_review_DF.printSchema()

root
 |-- MovieID: integer (nullable = true)
 |-- AvgRating: float (nullable = true)
 |-- TotalReviews: integer (nullable = true)



In [43]:
avg_review_DF.show(5)

+-------+---------+------------+
|MovieID|AvgRating|TotalReviews|
+-------+---------+------------+
|    144|3.3269231|          26|
|    150|   3.9025|         200|
|    168|3.1162791|          43|
|    186|    2.875|          48|
|    222|3.9318182|          22|
+-------+---------+------------+
only showing top 5 rows



# 5. Join Transformation on Two DataFrames
We want to join the avg_rating_total_review_DF with movies2_DF

In [44]:
joined_DF = avg_review_DF.join(movies2_DF,'MovieID', 'inner')

In [45]:
movies2_DF.printSchema()

root
 |-- MovieID: integer (nullable = true)
 |-- MovieTitle: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Genres_Array: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [46]:
joined_DF.printSchema()

root
 |-- MovieID: integer (nullable = true)
 |-- AvgRating: float (nullable = true)
 |-- TotalReviews: integer (nullable = true)
 |-- MovieTitle: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Genres_Array: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [47]:
joined_DF.show(6)

+-------+---------+------------+--------------------+--------------------+--------------------+
|MovieID|AvgRating|TotalReviews|          MovieTitle|              Genres|        Genres_Array|
+-------+---------+------------+--------------------+--------------------+--------------------+
|    144|3.3269231|          26|Brothers McMullen...|              Comedy|            [Comedy]|
|    150|   3.9025|         200|    Apollo 13 (1995)|Adventure|Drama|IMAX|[Adventure, Drama...|
|    168|3.1162791|          43| First Knight (1995)|Action|Drama|Romance|[Action, Drama, R...|
|    186|    2.875|          48|  Nine Months (1995)|      Comedy|Romance|   [Comedy, Romance]|
|    222|3.9318182|          22|Circle of Friends...|       Drama|Romance|    [Drama, Romance]|
|    300|     3.75|          90|    Quiz Show (1994)|               Drama|             [Drama]|
+-------+---------+------------+--------------------+--------------------+--------------------+
only showing top 6 rows



# 6. Filter DataFrame on an Array Column of DataFrame Using `array_contains`

## Exercise 8 (10 points)
Complete the following code to filter for Adventure movies.

In [48]:
from pyspark.sql.functions import array_contains
Adventure_DF = joined_DF.filter(array_contains("Genres_Array", \
                                               "Adventure")).select("MovieID","AvgRating","TotalReviews","MovieTitle")

In [49]:
Adventure_DF.show(7)

+-------+---------+------------+--------------------+
|MovieID|AvgRating|TotalReviews|          MovieTitle|
+-------+---------+------------+--------------------+
|    150|   3.9025|         200|    Apollo 13 (1995)|
|    480|3.7062044|         274|Jurassic Park (1993)|
|    552|     3.14|          50|Three Musketeers,...|
|    588|3.6744187|         215|      Aladdin (1992)|
|    720|      4.0|          45|Wallace & Gromit:...|
|     60| 2.828125|          32|Indian in the Cup...|
|   1884|    3.625|          44|Fear and Loathing...|
+-------+---------+------------+--------------------+
only showing top 7 rows



In [50]:
Adventure_DF.count()

1116

In [51]:
Sorted_Adventure_DF = Adventure_DF.orderBy('AvgRating', ascending=False)

In [52]:
Sorted_Adventure_DF.show(50)

+-------+---------+------------+--------------------+
|MovieID|AvgRating|TotalReviews|          MovieTitle|
+-------+---------+------------+--------------------+
|     53|      5.0|           1|     Lamerica (1994)|
|   7355|      5.0|           1|Mr. Toad's Wild R...|
|  92210|      5.0|           1|Disappearance of ...|
|  95313|      5.0|           1|Jack-Jack Attack ...|
| 126430|      5.0|           1|  The Pacific (2010)|
| 110873|      5.0|           1|Centenarian Who C...|
|   5244|      5.0|           1|Shogun Assassin (...|
|  80839|      5.0|           1|  Secretariat (2010)|
| 102666|      5.0|           1|Ivan Vasilievich:...|
|   2880|      5.0|           1|Armour of God (Lo...|
|   5264|      5.0|           1|Clockstoppers (2002)|
|   6725|      5.0|           1|Sgt. Pepper's Lon...|
|   3612|      5.0|           1|The Slipper and t...|
|  97057|      5.0|           1|     Kon-Tiki (2012)|
|   5301|      5.0|           1|Bite the Bullet (...|
|   4591|      5.0|         

In [53]:
Sorted_Adventure_DF.describe().show()

+-------+------------------+------------------+------------------+--------------------+
|summary|           MovieID|         AvgRating|      TotalReviews|          MovieTitle|
+-------+------------------+------------------+------------------+--------------------+
|  count|              1116|              1116|              1116|                1116|
|   mean|34887.717741935485|3.2298615851496284|19.728494623655912|                null|
| stddev| 42190.01946774694|0.8093798887312385|36.577661514336434|                null|
|    min|                 1|               0.5|                 1|'Hellboy': The Se...|
|    max|            162672|               5.0|               291|Zorba the Greek (...|
+-------+------------------+------------------+------------------+--------------------+



# Exercise 9 (10 points)
Complete the code below to show the histogram of the number of total reviews for adventure movies.

In [54]:
Sorted_Adventure_DF.select("TotalReviews").rdd.map(lambda row: row[0]).histogram([1, 3, 5, 10, 100, 300 ])

([1, 3, 5, 10, 100, 300], [384, 137, 156, 388, 51])

# Exercise 10 (10 points)
Use the histogram information in the previous exercise to select a threshold on the minumum number of reviews to be included in what you consider "top adventure movies". Use DataFrame method `where` or `filter` to find all adventure movies whose TotalReviews is more than x, where x is a threshold you have chosen based on the histogram.

In [55]:
Top_Adventure_DF = Sorted_Adventure_DF.where(col("TotalReviews") > 300 )

In [56]:
Top_Adventure_DF.show(10)

+-------+---------+------------+--------------------+
|MovieID|AvgRating|TotalReviews|          MovieTitle|
+-------+---------+------------+--------------------+
|   1948|4.4583335|          12|    Tom Jones (1963)|
|    969|     4.42|          50|African Queen, Th...|
|  32898|4.3333335|           6|Trip to the Moon,...|
|   2344|4.3333335|           6|Runaway Train (1985)|
|   7063|4.3333335|           9|Aguirre: The Wrat...|
|   1254|      4.3|          30|Treasure of the S...|
|   6016|4.2971015|          69|City of God (Cida...|
|   2019|4.2777777|          54|Seven Samurai (Sh...|
|    908| 4.270115|          87|North by Northwes...|
|   2936|     4.25|           6|Sullivan's Travel...|
+-------+---------+------------+--------------------+
only showing top 10 rows



In [57]:
Top_Adventure_DF.count()

546

## Exercise 11 (10 ponts)
Complete the code below to save the Adventure Movies, ordered by average rating, that received at least x reviews (x is the threshold you set for Exercise 9.

In [58]:
output_path = "/storage/home/hxw5245/Lab4/Lab4_Sorted_Top_Adventure_Movies"
Top_Adventure_rdd = Top_Adventure_DF.rdd
Top_Adventure_rdd.saveAsTextFile(output_path)

In [59]:
ss.stop()